In [44]:
import subprocess
result=subprocess.run(["sudo","cpupower","frequency-set","-u","6.0GHz"],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
if result.returncode==0:
    print("Success")
else:
    print("Failed")

Success


#### Breakdown of a single neural network
- X -> input
- Wx -> Weights
- bx -> bias
- A  -> Activation function
- Y -> Output

Z =W1.X +b1

Z' = A(Z)=W1.X+b1 , this node has an activation function

Y= W2.Z' + b2, this node has no activation function

- Loss functin:
- Caculate Gradient Using back propagation
- Optimizer 

### Components of pytorch
- Base class for defining custom model is `torch.nn.Module`

- Fully connected or dense layers `torch.nn.linear`

- Activation function `torch.nn.ReLU`

- Optimizer `torch.optim`

- Loss function `torch.nn.CrossEntropyLoss`

- Loads data in batch `torch.utils.data.DataLoader`


### Different way to create neural network

1. Functional:  Flexible, harder to interpret 
2. Sequential: nn.Sequential

### Building a neural network

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim

In [46]:
## functional API

class SimpleFunctionalNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleFunctionalNN,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

##### $$\text{Input} (x) \rightarrow \text{Linear Layer 1} \rightarrow \text{ReLU} \rightarrow \text{Linear Layer 2} \rightarrow \text{Output}$$

In [47]:
class SimpleSequentialNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleSequentialNN,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size)
        )

    def forward(self,x):
        return self.network(x)


### Training the neural network

In [48]:
model_fun1=SimpleFunctionalNN(input_size=4,hidden_size=8,output_size=3)

In [49]:
print(model_fun1)

SimpleFunctionalNN(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)


In [50]:
x=torch.randn(10,4) # 10 sample, 4 features
y=torch.randint(0,3,(10,))

In [51]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model_fun1.parameters(),lr=0.01)

In [52]:
print(x,y)

tensor([[-0.7840, -0.8075, -2.0283, -1.0192],
        [-1.7150, -0.2097, -0.7127,  2.4443],
        [-0.9118, -0.4373,  2.2787, -1.3370],
        [-0.3372, -0.9189, -0.3474,  1.1496],
        [-0.4623,  0.9344, -0.4068, -0.6980],
        [-1.4536,  0.3917,  1.1240,  0.7401],
        [ 1.3539, -2.7336, -0.2141,  1.6017],
        [ 0.2787, -0.3806, -1.8814,  0.6828],
        [ 1.5065, -0.0655, -0.4001, -0.2155],
        [ 1.3801,  0.0702,  0.6216, -0.1469]]) tensor([1, 2, 0, 1, 2, 2, 2, 2, 0, 0])


In [53]:
### training loop

epoch=40
for e in range(epoch):
    optimizer.zero_grad()
    outputs=model_fun1(x)
    loss=criterion(outputs,y)
    loss.backward()
    optimizer.step()
    if (e+1)%10==0:
       print(f"Epoch {e+1}/4000, Loss: {loss.item():.4f}")

Epoch 10/4000, Loss: 0.8472
Epoch 20/4000, Loss: 0.6302
Epoch 30/4000, Loss: 0.4494
Epoch 40/4000, Loss: 0.3168


In [54]:
import pandas as pd
import numpy as np

df=pd.read_csv("https://raw.githubusercontent.com/Arannamoy/datasets/refs/heads/main/Medical%20Cost%20Personal%20Datasets/insurance.csv")

df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [56]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [57]:
df.shape

(1338, 7)

In [58]:
df[df.duplicated()]

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.5631


In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [60]:
train_df,test_df=train_test_split(df,test_size=0.25,random_state=42)


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [62]:
print(df['sex'].unique())
print(df['smoker'].unique())
print(df['region'].unique())

['female' 'male']
['yes' 'no']
['southwest' 'southeast' 'northwest' 'northeast']


In [63]:
label_encoder={}
categorical_colums=["sex","smoker","region"]
for col in categorical_colums:
    le=LabelEncoder()
    train_df[col]=le.fit_transform(train_df[col].astype(str)) ### because encoder works on str not pd series
    test_df[col]=le.transform(test_df[col].astype(str))
    label_encoder[col]=le
    print(type(test_df[col]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [64]:
X_train=train_df.drop(columns=['charges'])
y_train=train_df['charges']
X_test=test_df.drop(columns=['charges'])
y_test=test_df['charges']

In [65]:
print(X_train.head(5))
print(y_test.head(5))

      age  sex     bmi  children  smoker  region
693    24    1  23.655         0       0       1
1297   28    0  26.510         2       0       2
634    51    1  39.700         1       0       3
1022   47    1  36.080         1       1       2
178    46    0  28.900         2       0       3
764      9095.06825
887      5272.17580
890     29330.98315
1293     9301.89355
259     33750.29180
Name: charges, dtype: float64


In [66]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [67]:
# Convert tensor
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.float32).view(-1,1)

In [68]:
y_train_tensor.shape

torch.Size([1003, 1])

In [69]:
### define neural network model
class SimpleNNLinearRegression(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleNNLinearRegression,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size),
            nn.ReLU(),
            nn.Linear(output_size,1)
        )

    def forward(self,x):
        return self.network(x)

In [70]:
simple_nn_linear_regression_model=SimpleNNLinearRegression(X_train_tensor.shape[1],64,128)

In [71]:
print(simple_nn_linear_regression_model)

SimpleNNLinearRegression(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [72]:
# loss and optimizer

criterion=nn.MSELoss()
optimizer=optim.Adam(simple_nn_linear_regression_model.parameters(),lr=0.01)


In [73]:
epochs=100000
for epoch in range(epochs):
    simple_nn_linear_regression_model.train()
    optimizer.zero_grad()
    predictions=simple_nn_linear_regression_model(X_train_tensor)
    loss=criterion(predictions,y_train_tensor)
    loss.backward()
    optimizer.step()

    if(epoch+1)%10000==0:
        print(f"epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")



epoch 9999/100000, Loss: 6413918.5000
epoch 19999/100000, Loss: 4691910.0000
epoch 29999/100000, Loss: 3989956.2500
epoch 39999/100000, Loss: 3609507.2500
epoch 49999/100000, Loss: 3349737.5000
epoch 59999/100000, Loss: 3203039.0000
epoch 69999/100000, Loss: 3067365.2500
epoch 79999/100000, Loss: 2968773.0000
epoch 89999/100000, Loss: 2834791.7500
epoch 99999/100000, Loss: 2719076.0000


In [74]:
### model evaluation
simple_nn_linear_regression_model.eval()
y_pred=simple_nn_linear_regression_model(X_test_tensor).detach().numpy()

In [75]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,root_mean_squared_error

In [76]:
mse=mean_squared_error(y_test_tensor.numpy(),y_pred)
mae=mean_absolute_error(y_test_tensor.numpy(),y_pred)
r2=r2_score(y_test_tensor.numpy(),y_pred)
rmse=mse**0.5
print(mse)
print(mae)
print(r2)
print(rmse)

73728824.0
5647.31201171875
0.5113773941993713
8586.549015757146


In [77]:
def predict_charges(age,sex,bmi,children,smoker,region):
    input_data=pd.DataFrame([[age,sex,bmi,children,smoker,region ]],columns=['age','sex','bmi','children','smoker','region'])
    for col in categorical_colums:
        input_data[col]=label_encoder[col].transform(input_data[col])
    input_data=scaler.transform(input_data)
    input_tensor=torch.tensor(input_data,dtype=torch.float32)
    predict_charges=simple_nn_linear_regression_model(input_tensor).item()
    return predict_charges

In [78]:
print(predict_charges(19,'female',27.9,0,"yes","southwest"))
print(predict_charges(60,'female',27.9,0,"yes","southwest"))

16857.345703125
41092.74609375


In [79]:
result=subprocess.run(["sudo","cpupower","frequency-set","-u","2.8GHz"],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
if result.returncode==0:
    print("Success")
else:
    print("Failed")

Success
